In [43]:
import threading

In [44]:
def decodelzw(compressed_data, rank):
    global NEXT_CODE
    decompressed_data = []
    string = ''
    print('|' * 10, f'HILO {rank} DECODIFICANDO -> {compressed_data}')
    for code in compressed_data:
        if code not in DICTIONARY:
            DICTIONARY[code] = string + (string[0])
        decompressed_data += DICTIONARY[code]
        if not(len(string) == 0):
            DICTIONARY[NEXT_CODE] = string + (DICTIONARY[code][0])
            NEXT_CODE += 1
        string = DICTIONARY[code]
    print('=' * 10, f'RESULTADO HILO {rank}  -> {decompressed_data}')
    DECODE_RESULT[rank] = decompressed_data

In [45]:
def parse_compressed_data(compressed_data):
    compressed_data = [i.split(',') for i in compressed_data.split('\n')]
    for i in range(len(compressed_data)):
        for j in range(len(compressed_data[i])):
            compressed_data[i][j] = int(compressed_data[i][j])
    return compressed_data

In [46]:
def open_compressed_data(file):
    file = open(file, 'r')
    compressed_data = file.read()
    parsed_compressed_data = parse_compressed_data(compressed_data)
    return parsed_compressed_data

In [47]:
# Paralelizar decodificacion
print('~' * 10, 'ABRIENDO ARCHIVO DE CODIFICACION')
RESULT = open_compressed_data('encoded_data.txt')
print('~' * 10, 'INICIALIZANDO DICCIONARIO CON 256 CARACTERES ASCII')
DICTIONARY_SIZE = 256
DICTIONARY = dict([(x, chr(x)) for x in range(DICTIONARY_SIZE)])
BITS = 9
MAXIMUM_SIZE = pow(2,int(BITS)) 
NEXT_CODE = 256

P = 4
DECODE_RESULT = [[] for p in range(P)]
print('~' * 10, f'UTILIZANDO {P} HILOS')

for rank in range(P):
    # Ejecutar en paralelo
    args = (RESULT[rank], rank)
    t = threading.Thread(target=decodelzw, args=args)
    t.start()
    t.join()

print('~' * 10, 'UNIFICANDO RESULTADOS DECODIFICADOS')
DECODE_RESULT = ''.join(sum(DECODE_RESULT, []))
print('~' * 10, f'RESULTADO FINAL DE DECODIFICACION => {DECODE_RESULT}')
print('~' * 10, 'GUARDANDO DE CODIFICACION')
output_file = open('decoded_data.txt', 'w+')
output_file.write(DECODE_RESULT)
output_file.close()

~~~~~~~~~~ ABRIENDO ARCHIVO DE CODIFICACION
~~~~~~~~~~ INICIALIZANDO DICCIONARIO CON 256 CARACTERES ASCII
~~~~~~~~~~ UTILIZANDO 4 HILOS
|||||||||| HILO 0 DECODIFICANDO -> [116, 104]
========== RESULTADO HILO 0  -> ['t', 'h']
|||||||||| HILO 1 DECODIFICANDO -> [256]
========== RESULTADO HILO 1  -> ['t', 'h']
|||||||||| HILO 2 DECODIFICANDO -> [101, 103]
========== RESULTADO HILO 2  -> ['e', 'g']
|||||||||| HILO 3 DECODIFICANDO -> [97, 109, 101]
========== RESULTADO HILO 3  -> ['a', 'm', 'e']
~~~~~~~~~~ UNIFICANDO RESULTADOS DECODIFICADOS
~~~~~~~~~~ RESULTADO FINAL DE DECODIFICACION => ththegame
~~~~~~~~~~ GUARDANDO DE CODIFICACION
